In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import csv
import sys
import pandas as pd
from ast import literal_eval
import re
import sys
sys.path.append("../")
import re
from ast import literal_eval
from rdfHandler import rdfObject
from sklearn import model_selection, naive_bayes, svm
import nltk
from nltk.corpus import wordnet
from nltk.corpus import brown
freqs = nltk.FreqDist(w.lower() for w in brown.words())
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
from nltk.corpus import stopwords
from collections import defaultdict
from collections import Counter
import string
brown = nltk.corpus.brown
stoplist = stopwords.words('english')

In [3]:
words = [word[0] for word in brown.tagged_words(categories="news")]
common_words = Counter(words).most_common(10000)[1000:10000]

In [4]:
training_set = []
antonyms = []

In [5]:
count = 0
for w in common_words:
    if "_" in w[0]:
        continue
    w = w[0].lower()
    count += 1
    for syn in wordnet.synsets(w.lower()):
        highest = 0
        highest_ant = 0
        for i in syn.lemmas():
            frequency = freqs[i.name()]
            if frequency > highest and "_" not in i.name() and i.name() != w:
                highest = frequency
                best = i.name()
            if i.antonyms():
                for i in i.antonyms():
                    frequency_ant = freqs[i.name()]
                    if frequency_ant > highest_ant and "_" not in i.name() and i.name() != w:
                        highest_ant = frequency_ant
                        best_ant = i.name()
    training_set.append((w, best))
    antonyms.append((w, best_ant))

In [6]:
print(len(training_set))

9000


In [7]:
antonyms

[('include', 'exclude'),
 ('force', 'push'),
 ("doesn't", 'push'),
 ('hands', 'push'),
 ('contract', 'expand'),
 ('appeared', 'disappear'),
 ('driving', 'attract'),
 ('lee', 'attract'),
 ('society', 'attract'),
 ('angeles', 'attract'),
 ('fourth', 'attract'),
 ('double', 'multivalent'),
 ('beat', 'multivalent'),
 ('players', 'multivalent'),
 ('finished', 'unfinished'),
 ('knee', 'unfinished'),
 ('happy', 'unhappy'),
 ('mickey', 'unhappy'),
 ('award', 'unhappy'),
 ('evening', 'unhappy'),
 ('music', 'unhappy'),
 ('driven', 'attract'),
 ('arrested', 'attract'),
 ('august', 'attract'),
 ('strike', 'miss'),
 ('gin', 'miss'),
 ('concert', 'miss'),
 ('independence', 'miss'),
 ('personnel', 'miss'),
 ('urged', 'miss'),
 ('continue', 'discontinue'),
 ('previous', 'discontinue'),
 ('these', 'discontinue'),
 ('elected', 'discontinue'),
 ('age', 'discontinue'),
 ('blue', 'discontinue'),
 ('bond', 'discontinue'),
 ('offered', 'discontinue'),
 ('veteran', 'discontinue'),
 ('leading', 'following'),
 

In [8]:
# Create an RDF object and read data from an RDF file available in the repository
rdf = rdfObject('https://mikeanders.org/data/Ontologies/DoD/DASD SKOS_Ontology.rdf', 'web')
all_labels = ['acronym', 'synonym', 'prefLabel', 'altLabel', 'broader', 'narrower', 'related']

In [9]:
# we're gonna get the label for each term in RDF
rdf_dict = {
    'acronym': [],
    'synonym': [],
    'prefLabel': [],
    'altLabel': [],
    'broader': [],
    'narrower': [],
    'related': []
}

In [10]:
#get all the rdf by rdf type
for label in all_labels:
    if label == 'acronym':
        set_list = set(rdf.acronymsList())
        for word in set_list:
            rdf_dict[label].append(word)
    elif label == 'synonym':
        set_list = set(rdf.synonymsList())
        for word in set_list:
            rdf_dict[label].append(word)
    else:
        set_list = set(rdf.customTagList(label))
        for word in set_list:
            rdf_dict[label].append(word)

In [11]:
# get the term and type
all_rdf = []
type_rdf = []
for label in all_labels:
    for word in rdf.customTagList(label):
        all_rdf.append(word)
        type_rdf.append(label)

In [12]:
# create dataframe
rdf_df = pd.DataFrame(zip(all_rdf, type_rdf), columns = ["Term", "Label"])
print(rdf_df['Label'].unique())
print("There are {} total terms in the RDF.".format(len(all_rdf)))

['acronym' 'synonym' 'prefLabel' 'altLabel' 'broader' 'narrower' 'related']
There are 9017 total terms in the RDF.


In [13]:
rdf_df.head()

,Term,Label
0,AB,acronym
1,A/D,acronym
2,AST,acronym
3,AO,acronym
4,ASC,acronym


In [14]:
synonyms = rdf_df[rdf_df['Label'] == 'synonym']

In [15]:
synonyms_list = synonyms['Term'].tolist()

In [16]:
model = KeyedVectors.load("custom_word_embeddings.bin")

In [17]:
model2 = KeyedVectors.load_word2vec_format("/Users/isaacstevens/nlp/GoogleNews-vectors-negative300.bin", binary=True)

In [18]:
for word in synonyms_list:
    try:
        model[word]
        print(word)
    except:
        continue

pallet
total item property record
military installation
freezer
refrigerator
allies
munitions
ordnance
antifreeze
stratification process
military installation
package
purchaser
exterior container
ration
repair part
ammunition
major end items
medical materiel
consumables
consumable
consumable
package
wartime
may
might
counterfeit materiel
counterfeit materiel
counterfeit materiel
package
critical safety item
critical safety item
demand
defective property
defective property
defective property
defective property
forecast
forecasting
demilitarization
depot maintenance
depot maintenance
disposition
disposal
storeroom
personal property
donees
donee
personnel
package
surplus
excess property
surplus property
surplus property
case
fax
facsimile
food cost index
food cost index
foreign purchased property
military installation
requisition
generator
generating activity
generating activity
government furnished equipment
government furnished property
government furnished equipment
government furnishe

In [19]:
synonym_gold_set = [('amazing', 'incredible'), 
               ('anger', 'enrage'), 
               ('angry', 'wrathful'), 
               ('answer', 'reply'), 
               ('awful', 'dreadful'),
              ('bad', 'depraved'),
              ('beautiful', 'gorgeous'),
              ('begin', 'start'),
              ('big', 'enormous'),
              ('brave', 'courageous'),
              ('break', 'fracture'),
              ('bright', 'sparkling'),
              ('calm', 'quiet'),
              ('come', 'approach'),
              ('crooked', 'bent'),
              ('base', 'fort'),
              ('box', 'container'),
              ('railroad', 'railway'),
              ('reallocation', 'redistribute'),
               ('must', 'should'),
               ('traceable', 'traceability'),
               ('wholesale', 'wholesale stock'),
               ('theater', 'theatre')
              ]

In [20]:
total_synonyms = training_set + synonym_gold_set

In [21]:
non_syn = [
    ('spoilage', 'salt'),
    ('uam', 'terminal'),
    ('point', 'out'),
    ('hot', 'cold'),
    ('trainee', 'actuarial'),
    ('unable', 'appraise'),
    ('formerly', 'folder'),
    ('types', 'aliens'),
    ('particles', 'nitrogen'),
    ('exchange', 'employee'),
    ('division', 'agents'),
    ('secreting', 'painting'),
    ('uam', 'terminal'),
    ('manifesting', 'baseband'),
    ('mechanism', 'daily'),
    ('unity', 'trucked'),
    ('penalty', 'holder'),
    ('center', 'business'),
    ('offender', 'identifier'),
    ('neck', 'inherent'),
    ('trespass', 'libel'),
    ('writing', 'animal'),
    ('tape', 'paper'),
    ('travelers', 'strategy'),
    ('recommendation', 'parties'),
    ('monthly', 'balance'),
    ('incident', 'enterprise'),
    ('meeting', 'airdrop'),
    ('month', 'day'),
    ('via', 'destination'),
    ('sustain', 'return'),
    ('organization', 'ineligibility'),
    ('ribbons', 'previously'),
    ('helicopter', 'guided'),
    ('enrollment', 'custody'),
    ('pins', 'motorized'),
    ('tuberculosis', 'giving'),
    ('toxicological', 'methodology'),
    ('wage', 'free'),
    ('spills', 'cable'),
    ('launcher', 'badges'),
    ('connor', 'anchorage'),
    ('elevate', 'corrects'),
    ('venipuncture', 'tripled'),
    ('orchard', 'apn'),
    ('spray', 'ducts'),
    ('salaries', 'employer'),
    ('radiological', 'programing'),
    ('particle', 'beta'),
    ('constraints', 'affordability'),
    ('mohawk', 'aos'),
    ('plantar', 'indeterminate'),
    ('yield', 'campaigning'),
    ('intimidating', 'casings'),
    ('xxxxxxxxxxxx', 'slave'),
    ('legacy', 'bagger'),
    ('replying', 'discharges'),
    ('recertification', 'handling'),
    ('swath', 'lots'),
    ('warfare', 'tactical'),
    ('sahara', 'juan'),
    ('skids', 'reinforced'),
    ('flt', 'bilirubin'),
    ('relating', 'language'),
    ('weapons', 'law'),
    ('trailers', 'artillery'),
    ('mil', 'hostile'),
    ('necrosis', 'drycleaning'),
    ('foundations', 'balsamic'),
    ('microorganisms', 'chlorinated'),
    ('lowermost', 'biomass'),
    ('watches', 'brass'),
    ('valvular', 'impurities'),
    ('rehearing', 'deliverables'),
    ('cloth', 'authorties'),
    ('pass', 'duty'),
    ('wooden', 'reactivity'),
    ('overshoe', 'jumper'),
    ('strengthening', 'drinking'),
    ('french', 'colonel'),
    ('sale', 'died'),
    ('pavements', 'converters'),
    ('underground', 'tanks'),
    ('transmissions', 'suggestion'),
    ('universal', 'trauma'),
    ('lactic', 'hardwood'),
    ('eldorado', 'cold'),
    ('platforms', 'fluorescent'),
    ('knowingly', 'chronic'),
    ('direction', 'appointment'),
    ('shares', 'prominently'),
    ('effecting', 'candidate'),
    ('nj', 'hampshire'),
    ('usage', 'receiving'),
    ('nepotism', 'sexually'),
    ('finality', 'abatement'),
    ('television', 'electric'),
    ('scans', 'implants'),
    ('variations', 'inadvertent'),
    ('bursting', 'imposing'),
    ('iodized', 'aqueous'),
    ('impact', 'full'),
    ('performance', 'materiel'),
    ('preliminary', 'defensive'),
    ('instructor', 'account'),
    ('handover', 'editor'),
    ('spray', 'chambers'),
    ('pretrial', 'entered'),
    ('teresa', 'brad'),
    ('nucleic', 'grease'),
    ('minimizing', 'discover'),
    ('radicular', 'greece'),
    ('coffee', 'camping'),
    ('grievous', 'great'),
    ('sonar', 'jamboree'),
    ('oxide', 'cartons'),
    ('rock', 'paintings'),
    ('ombudsperson', 'plumber'),
    ('prophecy', 'nicholson'),
    ('displaces', 'km'),
    ('mg', 'bring'),
    ('leaders', 'deemed'),
    ('screens', 'calculation'),
    ('cornea', 'compass'),
    ('cooling', 'apparatus'),
    ('specimen', 'accounts'),
    ('shadow', 'ops'),
    ('vacation', 'donations'),
    ('function', 'debriefings'),
    ('extensions', 'advice'),
    ('strategic', 'space'),
    ('visits', 'membership'),
    ('implants', 'desktop'),
    ('thickener', 'rolls'),
    ('offender', 'identifier'),
    ('tuberculosis', 'giving'),
    ('serum', 'assays'),
    ('foundations', 'balsamic'),
    ('pass', 'duty'),
    ('transitional', 'indemnity'),
    ('strengthening', 'drinking'),
    ('french', 'colonel'),
    ('direction', 'appointment'),
    ('scans', 'implants'),
    ('pretrial', 'entered'),
    ('spray', 'chambers'),
    ('loaders', 'arthralgia'),
    ('unserviceable', 'exposing'),
    ('grievous', 'great'),
    ('ombudsperson', 'plumber'),
    ('worsen', 'widow'),
    ('streamers', 'dust'),
    ('functional', 'commanders'),
    ('protections', 'exceptions'),
    ('sonar', 'jamboree'),
    ('tort', 'deregulatory'),
    ('scorched', 'sullivan'),
    ('pets', 'committees'),
    ('preparedness', 'assess'),
    ('reactivation', 'heartburn'),
    ('statutory', 'based'),
    ('siblings', 'lens'),
    ('cruelty', 'courtroom'),
    ('users', 'records'),
    ('concentrate', 'dust'),
    ('technically', 'malignant'),
    ('microphones', 'epoch'),
    ('sat', 'donating'),
    ('sizing', 'affecting'),
    ('life', 'consultation'),
    ('demonstration', 'confirmatory'),
    ('synthesized', 'launched'),
    ('benzimidazole', 'chips'),
    ('sierra', 'asi'),
    ('dix', 'detroit'),
    ('nw', 'avenue'),
    ('motors', 'missiles'),
    ('timing', 'mixtures'),
    ('drought', 'collapse'),
    ('research', 'deployments'),
    ('remote', 'connection'),
    ('migratory', 'bicycling'),
    ('line', 'entry'),
    ('garbage', 'coolers'),
    ('stamps', 'deed'),
    ('veterinarian', 'decommissioned'),
    ('indicator', 'broadcasts'),
    ('rockets', 'publishers'),
    ('prisms', 'loaders'),
    ('pesticides', 'fees'),
    ('variation', 'foreword'),
    ('orderlies', 'helicopter'),
    ('corsair', 'atlas'),
    ('gas', 'engines'),
    ('ave', 'abc'),
    ('moderate', 'cleanup'),
    ('vaults', 'grounding'),
    ('preferred', 'credits'),
    ('rights', 'replacement'),
    ('sectors', 'beach'),
    ('text', 'blank'),
    ('theory', 'darfur'),
    ('footnote', 'chaplaincy'),
    ('turnings', 'spar'),
    ('concrete', 'carbon'),
    ('retirements', 'participants'),
    ('officer', 'intelligence'),
    ('vault', 'fruit'),
    ('earthen', 'ek'),
    ('taking', 'accepts'),
    ('dinners', 'celebrations'),
    ('vectoring', 'thickener'),
    ('bm', 'aw'),
    ('rifle', 'fielded'),
    ('lo', 'scalding'),
    ('mechanical', 'guided'),
    ('wheezing', 'tunnel'),
    ('circuit', 'captions'),
    ('false', 'aggravated'),
    ('cohabit', 'desert'),
    ('viking', 'polaris'),
    ('presumption', 'exploitation'),
    ('medical', 'media'),
    ('incomplete', 'acknowledgment'),
    ('astronomy', 'multidisciplinary'),
    ('termination', 'inclusion'),
    ('docket', 'conversions'),
    ('unused', 'expenditure'),
    ('tomahawk', 'roofs'),
    ('farming', 'cues'),
    ('subsequently', 'copper'),
    ('charged', 'airworthiness'),
    ('caparisoned', 'opener'),
    ('ih', 'az'),
    ('irradiation', 'autonomous'),
    ('prisoner', 'campaign'),
    ('probe', 'insignia'),
    ('examine', 'endorses'),
    ('procurement', 'content'),
    ('hazard', 'capability'),
    ('swivel', 'monoplane'),
    ('fountain', 'certificate'),
    ('yet', 'power'),
    ('refrigerant', 'artifact'),
    ('infectivity', 'autoclave'),
    ('traders', 'amps'),
    ('summarization', 'compel'),
    ('roads', 'fences'),
    ('revalidation', 'activate'),
    ('priority', 'allowance'),
    ('resting', 'appearances'),
    ('pulmonary', 'classroom'),
    ('nerves', 'ganglia'),
    ('lines', 'creating'),
    ('immediately', 'explosive'),
    ('oman', 'china'),
    ('routes', 'occurs'),
    ('explains', 'allocates'),
    ('lethality', 'extend'),
    ('jamborees', 'ike'),
    ('osteopathic', 'biochemistry'),
    ('arrest', 'alarms'),
    ('collateral', 'capital'),
    ('technical', 'data'),
    ('sw', 'il'),
    ('measuring', 'dispenser'),
    ('having', 'diagram'),
    ('montana', 'iowa'),
    ('remitters', 'congregations'),
    ('woodworking', 'dance'),
    ('hoc', 'consultative'),
    ('voiding', 'incidentally'),
    ('discontinuing', 'advertising'),
    ('chromate', 'butts'),
    ('intercontinental', 'helicopter'),
    ('edgers', 'richardson'),
    ('osteopathic', 'crafts'),
    ('weaving', 'partitioning'),
    ('resides', 'consults'),
    ('persons', 'orders'),
    ('subornation', 'pacemaker'),
    ('shaw', 'pittsburgh'),
    ('afraid', 'bold'),
    ('killed', 'detained'),
    ('precluding', 'catastrophe'),
    ('narrators', 'fare'),
    ('visibility', 'notification'),
    ('plugs', 'mines'),
    ('woolen', 'deletion'),
    ('sheds', 'rails'),
    ('stroke', 'explosivity'),
    ('phones', 'imagers'),
    ('housekeeping', 'dps'),
    ('unapproved', 'solving'),
    ('ingesting', 'coke'),
    ('cloths', 'canister'),
    ('obtain', 'combinations'),
    ('tooth', 'bulldozers'),
    ('formulators', 'pumpers'),
    ('sg', 'aforementioned'),
    ('impairments', 'deactivation'),
    ('summarize', 'responsive'),
    ('lymphatic', 'edema'),
    ('dermatitis', 'crematory'),
    ('treasury', 'dropping'),
    ('hird', 'abe'),
    ('victimized', 'augmented'),
    ('secrets', 'identities'),
    ('forwards', 'evaluates'),
    ('screws', 'rods'),
    ('heating', 'electricity'),
    ('printing', 'amendments'),
    ('pathological', 'dance'),
    ('ft', 'blvd'),
    ('nanoscale', 'cabinetmakers'),
    ('informing', 'anorectal'),
    ('total', 'country'),
    ('reassigns', 'procures'),
    ('traders', 'schedulers'),
    ('searching', 'equation'),
    ('recurrent', 'diagnosis'),
    ('towing', 'fare'),
    ('softwood', 'hardwood'),
    ('detail', 'criteria'),
    ('spinal', 'ganglia'),
    ('disrupted', 'conducted'),
    ('transportability', 'bar'),
    ('regurgitation', 'humiliate'),
    ('processes', 'blood'),
    ('satisfies', 'conform'),
    ('receptionist', 'hidradenitis'),
    ('lt', 'gunnery'),
    ('rust', 'cigars'),
    ('seminary', 'alia'),
    ('underweight', 'village'),
    ('requires', 'accompany'),
    ('magnetos', 'entrenching'),
    ('online', 'accutane'),
    ('isolated', 'adaptation'),
    ('evacuation', 'alignment'),
    ('trigger', 'solicitations'),
    ('recognizing', 'fosters'),
    ('cyanide', 'chromium')
]

In [22]:
print(len(list(model2['crooked']) + list(model2['bent'])))

600


In [23]:
train_X = []
train_Y = []
for ele in total_synonyms:
    try:
        model2[ele[0]]
        model2[ele[1]]
        print(ele[0] + " - " + ele[1])
        train_X.append(list(model2[ele[0]]) + list(model2[ele[1]]))
        train_Y.append(1)
    except:
        continue

include - admit
force - storm
doesn't - storm
hands - hand
contract - cut
appeared - appear
driving - impulsive
lee - downwind
society - company
angeles - company
fourth - 4th
double - twice
beat - dead
players - player
finished - ruined
knee - stifle
happy - glad
mickey - glad
award - grant
evening - even
music - medicine
driven - determined
arrested - hold
august - revered
strike - coin
gin - noose
concert - noose
independence - noose
personnel - force
urged - urge
continue - cover
previous - premature
these - premature
elected - elect
age - mature
blue - dark
bond - tie
offered - offer
veteran - seasoned
leading - ahead
carry - expect
despite - contempt
rules - rule
bank - trust
you - trust
hard - heavily
plus - positive
an - positive
information - entropy
water - irrigate
de - irrigate
required - compulsory
immediately - now
joined - linked
corps - corporation
army - corporation
color - gloss
judge - try
build - establish
speed - accelerate
agency - way
basic - introductory
whole -

trend - cut
residents - resident
fire - fuel
apply - enforce
trucks - truck
frequently - often
consider - view
governor - regulator
grant - yield
review - retrospect
socialist - socialistic
resources - imagination
original - pilot
philosophy - doctrine
speaking - speak
cited - cite
local - cite
walter - cite
young - youthful
cocktail - youthful
stopped - stop
proud - majestic
challenge - dispute
stands - stand
twice - double
intentions - intention
seeking - seek
nomination - seek
chosen - choose
purchase - buy
edward - buy
experience - have
forest - timber
popular - pop
we'll - pop
green - gullible
rapidly - quickly
unless - quickly
presentation - introduction
paul - introduction
mates - felt
regarded - regard
ticket - fine
announce - herald
worse - bad
provides - provide
progress - build
loans - loan
interview - question
latter - question
advertising - push
weather - brave
ceremonies - ceremony
ride - mount
historic - historical
buildings - build
famous - noted
watch - see
located - s

parks - park
feature - sport
reynolds - sport
developing - underdeveloped
length - distance
flash - cheap
sons - boy
younger - young
flies - fly
explosive - volatile
target - place
active - dynamic
threatened - threaten
vincent - threaten
temporary - irregular
structure - construction
fear - reverence
waiting - wait
send - air
everybody - air
faith - religion
somebody - individual
circuit - circle
oregon - circle
grove - plantation
lodge - accommodate
stronger - strong
includes - include
motions - motion
striking - prominent
over - o'er
oriole - o'er
hyde - o'er
glove - o'er
pitchers - pitcher
ed - pitcher
chores - job
palm - handle
purchased - buy
milwaukee - buy
triumph - rejoice
jackie - rejoice
gentile - pagan
threw - throw
triple - threefold
streak - bar
drove - drive
weight - angle
camp - tent
carrying - expect
improved - better
rookie - better
art - artistry
manager - coach
coaches - coach
damage - wrong
wing - fly
clint - fly
seventh - 7th
nearing - near
self - ego
big - large


cookies - cookie
watching - see
motel - see
heat - wake
adults - adult
myself - adult
marvel - wonder
proprietor - owner
officiated - function
rhodes - function
kappa - function
freeman - function
skirt - ring
maid - maiden
conrad - maiden
kay - maiden
angelo - maiden
earl - maiden
fun - play
promises - promise
we've - promise
hats - hat
phase - form
flowers - flower
handsome - big
typical - distinctive
clothing - robe
household - house
bundles - bundle
jewish - bundle
contribution - donation
men's - donation
aides - help
lewis - help
exhibit - march
monte - march
meyer - march
chairmen - president
easter - president
louise - president
carnival - fair
margaret - fair
auditorium - fair
ladies - lady
queen - pansy
slim - slender
silk - slender
simpson - slender
decline - correct
affect - move
cab - taxi
mount - ride
store - storage
negroes - storage
burned - burnt
arrival - comer
recording - record
boards - board
ambulance - board
dispatched - dispatch
slowly - easy
tires - bore
trained 

voice - sound
wouldn't - sound
everyone - sound
statewide - sound
mothers - get
empire - get
speakers - speaker
incumbent - speaker
seeks - seek
vacated - lift
trends - cut
editing - edit
denomination - designation
reads - read
emphasizes - emphasize
lord - noble
forthcoming - coming
breakthrough - find
babies - baby
immigrant - baby
accomplish - reach
meets - meet
engaging - piquant
pioneer - initiate
lending - lend
fraud - fake
dismissal - release
mistrial - release
acquittal - release
conspirators - release
philip - release
proof - validation
defraud - con
winless - con
athletics - sport
frames - frame
fisher - fisherman
righthander - fisherman
middle - center
scoreless - hitless
norm - average
eighth - 8th
throws - throw
dirt - scandal
catcher - scandal
rookie - scandal
baseman - scandal
performed - do
appearance - show
pete - show
ward - guard
failing - fail
bunt - butt
popped - pop
barker - pop
crossed - cross
tallies - total
grabbed - grab
error - mistake
stretch - extend
blow -

gee - g
friend - champion
buried - interred
marks - mark
page - mark
innocent - free
widowed - widow
bailey - widow
criminal - felonious
promptly - readily
clients - guest
designing - scheming
nowhere - scheming
sharing - share
generation - propagation
much - often
polaris - often
branch - separate
navy - separate
fastest - quickest
destroyers - destroyer
ships - ship
calling - call
indictments - indictment
narcotic - indictment
peddlers - peddler
posed - get
rivalries - competition
nominated - name
medal - palm
sears - sear
indiana - sear
winston - sear
stewart - pull
cta - pull
bag - pocket
paycheck - pocket
express - extract
accosted - hook
fled - fly
blanche - fly
dunkel - fly
slaying - off
salesman - off
pursuit - interest
speeding - speed
skidded - slide
prairie - slide
anna - slide
cell - cadre
woman's - cadre
removed - distant
still - even
misfortune - even
arson - even
cash - even
motion - gesture
deliberation - slowness
foreman - chief
definition - chief
packed - jammed
holy 

devotion - veneration
satisfying - comforting
comradeship - camaraderie
had - have
wasteful - uneconomical
duplication - duplicate
pitifully - duplicate
qualifications - reservation
specialized - specialize
brand - post
professors - professor
hiring - take
interests - interest
paradox - interest
grips - grip
caliber - bore
declines - decline
civil - civic
depreciation - disparagement
exaggerate - amplify
substitute - ersatz
actual - factual
artificial - contrived
structured - integrated
argue - indicate
allegedly - indicate
disputes - dispute
equivalent - tantamount
abuses - abuse
rationale - principle
marginal - bare
implicit - bare
theory - possibility
managements - management
penalties - penalty
fundamental - profound
unprecedented - profound
rational - intellectual
commodities - good
bags - bag
agricultural - farming
parliament - farming
combining - flux
mechanics - mechanic
necessity - necessary
monopolistic - necessary
measures - value
aimed - aim
gore - bloodshed
erupted - erupt

hastened - rush
analysis - psychoanalysis
setbacks - blow
exploratory - blow
vagueness - blow
tactical - blow
semipublic - blow
accredited - licensed
popularity - licensed
tentative - doubtful
exploratory - doubtful
enunciate - articulate
scale - surmount
reiterated - repeat
profound - unplumbed
cornerstone - basis
solemnly - basis
repeated - perennial
affirmation - statement
conflict - difference
rekindling - difference
realization - fruition
wherever - fruition
globe - ball
essence - perfume
geographical - geographic
impetus - impulse
directions - direction
cooperation - direction
underdeveloped - developing
deterrent - check
haltingly - check
erupts - erupt
docile - gentle
inspection - review
cabinet - locker
facto - locker
modifications - change
compelled - compel
excuses - excuse
confrontations - confrontation
arises - rise
aggression - hostility
former - other
conciliatory - flexible
gallup - flexible
understates - minimize
restrain - intimidate
amateurish - amateur
monumental - 

intervened - intervene
preliminary - overture
overhauling - overhaul
wide - widely
repairs - repair
sons - boy
cambridge - boy
can't - boy
hooked - dependent
transit - passage
certain - sure
occupancy - occupation
testifies - show
accomodations - show
councilwoman - show
pedigreed - pedigree
males - male
females - female
replies - answer
penalized - punish
leary - punish
regulate - baffle
catchers - catcher
bites - bite
backs - back
indorsed - endorse
compensated - salaried
licenses - license
inspections - review
barnet - review
lieberman - review
petitions - petition
hess - petition
victim - petition
reprimanded - lecture
violating - violate
redevelopers - violate
area - expanse
bounded - spring
dicks - tool
garden - tool
metal - metallic
builder - metallic
west - westward
bar - banish
reconstruction - banish
planner - banish
sectors - sector
barred - bar
vehicular - bar
landscaped - landscape
walkways - walk
eliminates - eliminate
severly - eliminate
tore - tear
battalion - pack
repr

sporting - betting
sweets - sweet
readjust - sweet
vast - huge
safari - campaign
bleak - cutting
vacancy - vacuum
suitcases - bag
comic - funny
reared - raise
grandma - grandmother
summertime - summer
onslaught - barrage
influx - inflow
testify - show
scots - show
heather - broom
wagons - wagon
roadside - wayside
restaurants - restaurant
souvenir - memento
snake - serpent
boasting - feature
tastes - taste
peculiarities - curiosity
advertises - push
vented - air
carpeted - carpet
innumerable - infinite
swimming - liquid
hospitable - liquid
poised - collected
brink - verge
motorists - motorist
tourists - tourist
accommodated - accommodate
village - hamlet
luxury - hamlet
boxwood - box
accidentally - unintentionally
carpeting - carpet
bath - bathe
towels - towel
tab - pill
judith - pill
ellen - pill
ferris - pill
bridegroom - groom
monica - groom
ucla - groom
theology - divinity
attendants - attendant
sandra - attendant
sequoia - redwood
frances - redwood
elvis - acid
gamma - acid
mortar 

In [24]:
for ele in (non_syn + antonyms):
    try:
        model2[ele[0]]
        model2[ele[1]]
        print(ele[0] + " - " + ele[1])
        train_X.append(list(model2[ele[0]]) + list(model2[ele[1]]))
        train_Y.append(0)
    except:
        continue

spoilage - salt
point - out
hot - cold
trainee - actuarial
unable - appraise
formerly - folder
types - aliens
particles - nitrogen
exchange - employee
division - agents
secreting - painting
manifesting - baseband
mechanism - daily
unity - trucked
penalty - holder
center - business
offender - identifier
neck - inherent
trespass - libel
writing - animal
tape - paper
travelers - strategy
recommendation - parties
monthly - balance
incident - enterprise
meeting - airdrop
month - day
via - destination
sustain - return
organization - ineligibility
ribbons - previously
helicopter - guided
enrollment - custody
pins - motorized
tuberculosis - giving
toxicological - methodology
wage - free
spills - cable
launcher - badges
connor - anchorage
elevate - corrects
venipuncture - tripled
orchard - apn
spray - ducts
salaries - employer
radiological - programing
particle - beta
constraints - affordability
mohawk - aos
plantar - indeterminate
yield - campaigning
intimidating - casings
xxxxxxxxxxxx - slave

career - unfairly
attended - unaccompanied
largest - little
candidates - little
hold - disagree
unit - disagree
assistant - disagree
begin - end
let - forbid
roads - forbid
yet - forbid
itself - forbid
november - forbid
increased - decreased
harry - decreased
death - birth
post - birth
stocks - birth
introduced - birth
budget - birth
joe - birth
adopted - native
coast - native
taxes - native
paris - native
gas - native
ended - begin
methods - begin
junior - senior
request - senior
outside - inside
actually - inside
facilities - inside
knowledge - inside
change - stay
step - stay
changes - stay
track - stay
leadership - stay
points - stay
treated - untreated
ground - figure
hopes - despair
nixon - despair
southeast - despair
prepared - unprepared
labor - unprepared
advantage - disadvantage
status - disadvantage
vital - disadvantage
things - disadvantage
placed - divest
retired - advance
product - advance
manufacturers - advance
goal - advance
station - advance
sell - buy
administration 

royal - still
certainly - still
factor - still
arkansas - still
encouraging - discouraging
trend - discouraging
residents - nonresident
fire - hire
apply - exempt
trucks - exempt
frequently - rarely
consider - rarely
governor - rarely
grant - deny
review - deny
socialist - capitalistic
resources - capitalistic
original - capitalistic
philosophy - capitalistic
speaking - capitalistic
cited - capitalistic
local - general
walter - general
young - old
cocktail - old
stopped - begin
proud - humble
challenge - humble
stands - yield
twice - yield
intentions - yield
seeking - yield
nomination - yield
chosen - yield
purchase - sell
edward - sell
experience - inexperience
forest - inexperience
popular - unpopular
we'll - unpopular
green - ripe
rapidly - slowly
unless - slowly
presentation - slowly
paul - slowly
mates - disentangle
regarded - disentangle
ticket - disentangle
announce - disentangle
worse - good
provides - good
progress - recede
loans - borrow
interview - borrow
latter - former
adv

depending - uninteresting
bids - uninteresting
ranged - uninteresting
engineering - uninteresting
director - uninteresting
sue - uninteresting
bankruptcy - uninteresting
behalf - uninteresting
widow - uninteresting
pleaded - uninteresting
tract - uninteresting
parks - uninteresting
feature - lack
reynolds - lack
developing - lack
length - lack
flash - lack
sons - girl
younger - old
flies - old
explosive - old
target - old
active - inactive
threatened - inactive
vincent - inactive
temporary - permanent
structure - permanent
fear - permanent
waiting - permanent
send - permanent
everybody - permanent
faith - permanent
somebody - permanent
circuit - permanent
oregon - permanent
grove - permanent
lodge - dislodge
stronger - impotent
includes - exclude
motions - exclude
striking - miss
over - miss
oriole - miss
hyde - miss
glove - miss
pitchers - miss
ed - miss
chores - miss
palm - miss
purchased - sell
milwaukee - sell
triumph - defeat
jackie - defeat
gentile - defeat
threw - defeat
triple 

voluntarily - involuntarily
bigger - little
file - little
emphasize - little
manhattan - little
publicly - privately
apparently - privately
assumed - privately
interviews - privately
confirm - negate
hunter - negate
taxi - negate
remarks - negate
influence - negate
secret - negate
aroused - sedate
petty - sedate
appears - disappear
languages - disappear
middle - early
question - answer
confusion - answer
happen - answer
missionary - answer
discovered - answer
owner - answer
classical - answer
gay - answer
indonesia - answer
pulled - repel
look - back
estimates - back
experts - back
filling - empty
eighteen - empty
economist - empty
fifteen - empty
maurice - empty
meanwhile - empty
slowed - accelerate
princess - accelerate
principles - accelerate
search - accelerate
succeeds - precede
artist - precede
miles - precede
begins - end
great - end
returning - end
bordering - end
battle - end
branches - end
johnson - end
guard - end
reserve - end
helm - end
surrounded - end
keeps - break
stead

...

best - badly
showman - badly
holders - badly
mets - badly
gate - badly
via - badly
nobel - badly
collect - spread
me - spread
everywhere - spread
roles - spread
opens - close
swim - sink
tells - sink
jean - sink
cheer - complain
gods - complain
patio - complain
bowman - complain
grandparents - complain
palace - complain
cocktails - complain
juniors - complain
decorations - complain
stanley - complain
bradford - complain
cheery - complain
qualities - complain
asset - liability
salvation - liability
civilian - military
tasks - military
bridge - military
alcoholics - military
parolees - military
pressing - military
solved - unsolved
coffee - unsolved
sugar - unsolved
vacation - unsolved
lovely - unsolved
magic - unsolved
ah - unsolved
journey - unsolved
tucson - unsolved
stations - unsolved
braced - sedate
vacationing - sedate
accommodations - sedate
maybe - sedate
shamrock - sedate
cellar - sedate
boast - sedate
suite - sedate
elegant - sedate
brocade - sedate
tile - sedate
ants - sedate

literature - morality
person - morality
exploring - morality
language - morality
austere - morality
ha - morality
windshield - morality
stuck - unstuck
smallest - large
get - end
unhappily - happily
doormen - happily
politely - happily
calm - stormy
sofas - stormy
undertaken - stormy
redecoration - stormy
unveiled - veiled
match - disagree
delighted - disagree
welcoming - disagree
hail - disagree
tent - disagree
listening - disagree
carolina - disagree
visitors - disagree
noticed - unnoticed
impressed - unnoticed
masons - unnoticed
historian - unnoticed
laying - unnoticed
wreath - unnoticed
separately - unnoticed
outsiders - unnoticed
attitude - unnoticed
exchanges - unnoticed
confrontation - unnoticed
desirable - undesirable
implement - exempt
announcements - exempt
lull - agitate
camp - agitate
tended - agitate
propaganda - agitate
aims - agitate
confronted - avoid
delicate - rugged
shifting - rugged
toughness - rugged
debates - rugged
illusion - rugged
ideally - rugged
satisfy - rug

contributing - trust
defeat - victory
effectiveness - ineffectiveness
inconsistencies - consistency
determining - consistency
muster - consistency
reconvenes - consistency
normalcy - consistency
terminate - hire
towel - hire
politicos - hire
acknowledge - deny
ineffectual - effective
underlying - effective
passage - effective
consonance - effective
acute - chronic
dilemma - chronic
constitution - chronic
forbids - permit
introduction - permit
inflate - permit
constant - permit
stab - permit
raiser - permit
unpleasant - pleasant
alternatives - pleasant
arouse - sedate
trims - sedate
exert - sedate
receptive - sedate
economize - waste
allegations - waste
imprudently - prudently
combat - prudently
elevated - lower
contracting - expand
estimates - expand
shortcuts - expand
unnoticed - noticed
erection - noticed
impossibly - possibly
legitimate - illegitimate
shortcuts - illegitimate
rigging - illegitimate
covering - uncover
transportation - uncover
signature - uncover
property - uncover
vo

balanced - unbalanced
lineman - unbalanced
jock - unbalanced
sutherland - unbalanced
quit - enter
polytechnic - enter
troy - enter
rejoining - enter
auspiciously - enter
engrossed - enter
cufflinks - enter
shaped - enter
dais - enter
columnist - enter
good - ill
meritorious - ill
sid - ill
lampoon - ill
skit - ill
imminent - ill
named - ill
relentlessly - ill
departure - ill
disappointments - ill
convince - ill
branch - ill
lease - ill
prominently - ill
nori - ill
sands - ill
georgetown - ill
writers - ill
saluted - ill
cup - ill
dublin - ill
amateur - professional
gardner - professional
tee - professional
charity - professional
golfers - professional
comeback - professional
twelve - professional
heralded - professional
rochester - professional
athlete - professional
endeared - professional
duffer - professional
flubbed - professional
monstrous - professional
brooding - professional
incompetents - professional
meditating - professional
abandonment - professional
despair - hope
paragon 

principally - unloading
necessitated - unloading
regrettable - unloading
supervisors - unloading
sufficient - insufficient
furlough - insufficient
furloughed - insufficient
thug - insufficient
robbing - insufficient
wiley - insufficient
bandit - insufficient
assailant - insufficient
choking - insufficient
boy - girl
mile - girl
mechanically - girl
cites - girl
memory - girl
hate - love
permanent - love
commissioners - love
zoning - love
misunderstandings - love
notes - ignore
doctors - break
partially - wholly
paralyzed - wholly
barber - wholly
deficiencies - adequacy
fifth - adequacy
salting - adequacy
crews - adequacy
storms - adequacy
conceding - adequacy
equipment - adequacy
snowfall - adequacy
halting - start
sent - start
manual - automatic
work - idle
banned - idle
chains - idle
admitting - exclude
overlooked - exclude
merchants - exclude
survive - succumb
recounting - succumb
observations - succumb
clearance - succumb
inefficient - efficient
poorly - well
blades - well
layer - w

In [25]:
print(len(train_X))

14558


In [26]:
count_0 = 0
count_1 = 0
for val in train_Y:
    if val == 0:
        count_0 += 1
    else:
        count_1 += 1
print(count_0)
print(count_1)

7482
7076


In [27]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.01, random_state=42)

In [28]:
SVM = svm.SVC(C=20, kernel='rbf', class_weight = {0: 0.9, 1: 0.1})

In [29]:
SVM.fit(X_train,y_train)

SVC(C=20, class_weight={0: 0.9, 1: 0.1})

In [30]:
y_test_pred = SVM.predict(X_test)
cm_test = confusion_matrix(y_test, y_test_pred)

In [31]:
cm_test

array([[81,  7],
       [10, 48]])

In [32]:
def cm_accuracy(cm):
    return (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][0] + cm[0][1] + cm[1][1])

In [33]:
cm_accuracy(cm_test)

0.8835616438356164

In [34]:
possible_synonyms = pd.read_csv("first_synonyms.csv", index_col=0)

In [35]:
possible_synonyms.count()

Second Term    155380
First Term     155366
dtype: int64

In [36]:
count = 0
positive_synonyms = []
count = 0
for index, row in possible_synonyms.iterrows():
    count += 1
    try:
        first = row['First Term'].lower()
        second = row['Second Term'].lower()
        vector = [list(model2[first]) + list(model2[second])]
        pred = SVM.predict(vector)
        if pred == 1:
            print("(" + "\'" + first + "\'" + ", " + "\'" + second + "\'" + ")")
            positive_synonyms.append((first, second))
            count += 1
    except:
        continue

('reagents', 'paint')
('misconduct', 'crimes')
('tile', 'fiberboard')
('space', 'facility')
('forensic', 'academic')
('purifiers', 'humidifiers')
('masturbatory', 'masturbate')
('medal', 'certificate')
('terrier', 'bulldog')
('treatment', 'handling')
('modifying', 'disestablishing')
('referrals', 'findings')
('packaged', 'loaded')
('requirement', 'needs')
('hive', 'bee')
('permitted', 'authorized')
('reduce', 'mitigate')
('seminary', 'alia')
('isolated', 'adaptation')
('demodulation', 'capsule')
('standoff', 'enzyme')
('staff', 'chiefs')
('networking', 'feasibility')
('employer', 'compilation')
('restating', 'paraphrasing')
('saturation', 'dug')
('unexploded', 'proving')
('bonuses', 'atlantic')
('windows', 'corporate')
('offense', 'life')
('possession', 'interest')
('skin', 'prostate')
('overseen', 'augmented')
('reissuance', 'formation')
('forcible', 'fondling')
('vessels', 'accountable')
('programs', 'activities')
('posters', 'impressed')
('than', 'marijuana')
('seriousness', 'seizur

('goals', 'funding')
('bs', 'bj')
('stock', 'postage')
('tn', 'protestant')
('parte', 'monte')
('inflicted', 'bodily')
('overage', 'depicting')
('campgrounds', 'rv')
('greens', 'turkey')
('crashed', 'clamshell')
('principles', 'analysis')
('meninges', 'doppler')
('taxability', 'screens')
('subcommittee', 'automated')
('landing', 'apprenticeship')
('premiums', 'passengers')
('wildland', 'seeking')
('keepers', 'blue')
('october', 'march')
('interstate', 'billings')
('storage', 'analysis')
('refused', 'mistake')
('theological', 'baccalaureate')
('partial', 'interest')
('sacked', 'pouched')
('leaflet', 'dispenses')
('agreements', 'administration')
('project', 'designated')
('nonjudicial', 'martial')
('primitive', 'fare')
('partnership', 'drug')
('obey', 'illustrate')
('tiger', 'corsair')
('guide', 'accuracy')
('psychoactive', 'narcotics')
('doc', 'affects')
('porches', 'propeller')
('transactions', 'computer')
('railcar', 'powder')
('physics', 'chemistry')
('opaque', 'instant')
('sorter', 

('redress', 'murder')
('suitability', 'disposal')
('clutches', 'sterilizer')
('or', 'in')
('specially', 'dispenser')
('meeting', 'carrier')
('picnic', 'halls')
('fruits', 'cotton')
('clear', 'achievement')
('precluding', 'flight')
('wing', 'helicopter')
('school', 'how')
('stains', 'lubricating')
('stocking', 'maintaining')
('respons', 'oscillator')
('monkeys', 'likelihood')
('enciphered', 'oak')
('swivel', 'fan')
('medication', 'elevator')
('prioritizes', 'consolidates')
('shipment', 'involvement')
('redactions', 'petitions')
('coverings', 'cigarettes')
('permanently', 'captured')
('brown', 'moro')
('recipient', 'cause')
('visitors', 'trial')
('mitigate', 'minimize')
('notification', 'accreditation')
('respondent', 'forecasts')
('colitis', 'coronary')
('supplies', 'opportunity')
('non', 'cleaning')
('resources', 'projects')
('cribs', 'baby')
('smoke', 'pots')
('substitution', 'event')
('disputed', 'containing')
('reconciliation', 'elections')
('status', 'condition')
('trade', 'lieu')


('maximizes', 'apprise')
('exploitation', 'because')
('depletion', 'absorption')
('censure', 'brook')
('hook', 'pasta')
('technician', 'helicopter')
('communicate', 'chart')
('solvent', 'netting')
('woodworkers', 'anatomy')
('teletype', 'switches')
('typhoons', 'cucumbers')
('removes', 'discusses')
('integrated', 'growth')
('stallion', 'dummy')
('corrodes', 'burners')
('shepherd', 'bullet')
('looting', 'carjacking')
('sunny', 'charles')
('preliminary', 'extension')
('torpedoes', 'magneto')
('interpreter', 'habitat')
('scrapers', 'excavators')
('cords', 'bulging')
('ive', 'arthritis')
('formation', 'discovery')
('prioritizing', 'justification')
('fragment', 'bk')
('whom', 'gov')
('host', 'event')
('comforters', 'chewable')
('sale', 'return')
('persons', 'identify')
('sanitation', 'explosion')
('endorses', 'briefs')
('operated', 'centralized')
('wage', 'explosives')
('scrapers', 'hardboard')
('grinders', 'iron')
('sporting', 'fiber')
('foundations', 'athletics')
('sheeting', 'cerebral')


('signaling', 'aperture')
('newer', 'hourglass')
('vice', 'directory')
('lightweight', 'injection')
('marijuana', 'assault')
('illustrate', 'burn')
('negotiation', 'clear')
('privately', 'appropriated')
('stemming', 'earths')
('toiletries', 'ashes')
('cesspool', 'lavatories')
('waiver', 'elements')
('probationary', 'baggage')
('guns', 'automotive')
('sandwich', 'gn')
('distribute', 'collect')
('xxxxxxxxx', 'third')
('opinion', 'content')
('tower', 'showerheads')
('taxability', 'notifications')
('distributive', 'blended')
('competitive', 'beleaguered')
('panel', 'mailer')
('testimony', 'negotiate')
('flow', 'computation')
('rl', 'arresting')
('entitlements', 'benefits')
('threats', 'comparing')
('wv', 'nova')
('delta', 'dart')
('supplementary', 'driving')
('jewish', 'aden')
('thigh', 'sulfuric')
('lights', 'flares')
('laundries', 'arbitrations')
('strip', 'mitigating')
('on', 'decision')
('grazing', 'antiquities')
('injurious', 'eruption')
('rifle', 'calculation')
('launchers', 'helicop

...

('nominations', 'discovery')
('flashpoint', 'decks')
('fishing', 'cottages')
('churches', 'uss')
('pencil', 'cds')
('fittings', 'couplings')
('permanent', 'acumen')
('translate', 'generates')
('narrowband', 'destroyers')
('demilitarizing', 'bicuspid')
('valley', 'fixing')
('healthy', 'apprehending')
('drums', 'blankets')
('shipment', 'inspections')
('forged', 'certifier')
('trinitrotoluene', 'movie')
('means', 'examples')
('operator', 'community')
('estimates', 'burden')
('that', 'mil')
('invites', 'desired')
('forces', 'corps')
('_______________', 'mock')
('lutheran', 'episcopal')
('masonry', 'doppler')
('republic', 'puerto')
('spines', 'nod')
('recruitment', 'goals')
('derivatives', 'hat')
('behaviors', 'behavior')
('divert', 'abstraction')
('sturdy', 'cream')
('granite', 'airfield')
('requesting', 'member')
('terminate', 'misbehave')
('rock', 'bells')
('prevent', 'disclose')
('dynamic', 'calculation')
('waivers', 'validation')
('overwritten', 'luggage')
('liabilities', 'flight')
('r

('tennessee', 'hill')
('sponsorship', 'impact')
('lbs', 'ft')
('platform', 'lists')
('deliverables', 'composition')
('scratching', 'melting')
('new', 'direct')
('retail', 'badge')
('straps', 'smelting')
('floats', 'imprints')
('segregation', 'redistribution')
('safeguarding', 'motor')
('thigh', 'elevator')
('tanker', 'port')
('sinuses', 'constituents')
('indicators', 'construction')
('winds', 'embers')
('initial', 'based')
('served', 'intercourse')
('p.m.', 'a.m.')
('transparency', 'cancellation')
('overseas', 'honors')
('shipped', 'delivered')
('managers', 'goal')
('streaker', 'maverick')
('upon', 'accounts')
('parsed', 'powered')
('hawkeye', 'bronco')
('taken', 'providing')
('titling', 'currency')
('cures', 'antiquities')
('subscale', 'monoplane')
('intake', 'company')
('headgear', 'blankets')
('monsignor', 'aas')
('seymour', 'canal')
('catholic', 'anglican')
('optimizes', 'allocate')
('search', 'originator')
('loanee', 'timestamp')
('workstation', 'kevlar')
('_______________________

('wv', 'denver')
('trivalent', 'tea')
('misusing', 'administering')
('milli', 'rotax')
('final', 'aircraft')
('strait', 'hill')
('included', 'badge')
('publication', 'infrastructure')
('torpedoes', 'grenades')
('priority', 'exchange')
('spectrum', 'electronically')
('quarantining', 'halls')
('request', 'assistance')
('af', 'aa')
('recycle', 'quotations')
('councils', 'acting')
('liters', 'ft')
('tattoo', 'astronaut')
('range', 'frequency')
('minimal', 'maintaining')
('road', 'box')
('vertical', 'lapel')
('explosives', 'base')
('undefined', 'tailoring')
('hostile', 'acronym')
('lamp', 'enlarged')
('synchronize', 'prioritize')
('soft', 'inserts')
('popcorn', 'responder')
('doctoral', 'baccalaureate')
('mutilation', 'advice')
('toluene', 'spikes')
('sta', 'classical')
('channels', 'briefing')
('resist', 'humiliate')
('shredded', 'crumbled')
('before', 'after')
('suicide', 'checklist')
('transfer', 'means')
('baseband', 'ashes')
('invoking', 'conspiracy')
('robbery', 'nonfatal')
('caseload

('employee', 'behalf')
('peas', 'grilled')
('premixed', 'drummed')
('removal', 'payment')
('propagating', 'vessel')
('review', 'reporting')
('detection', 'approach')
('goals', 'composition')
('malice', 'bicycling')
('securities', 'global')
('profits', 'dividends')
('searched', 'scanned')
('territorial', 'pools')
('pylons', 'declaring')
('buttons', 'algae')
('reenlistments', 'incidentally')
('cockfighting', 'representations')
('ceramics', 'autobiography')
('grease', 'granite')
('mines', 'bombs')
('restrictions', 'notifications')
('exchange', 'commissary')
('_______________________________________________________', 'noun')
('refrains', 'gleaned')
('floppy', 'bob')
('unfavorably', 'dessert')
('straight', 'muzzle')
('solvent', 'dust')
('compass', 'azimuth')
('seals', 'mediation')
('petition', 'claim')
('persona', 'histogram')
('countering', 'combating')
('venting', 'mutilation')
('landing', 'filled')
('smoke', 'oxygen')
('rigid', 'fans')
('sweet', 'droplet')
('slag', 'glint')
('paints', 'c

('cl', 'bj')
('screenwriting', 'playwriting')
('marrow', 'dura')
('responsible', 'participating')
('marshes', 'kilns')
('sewage', 'gasoline')
('rum', 'quartermaster')
('sexually', 'forgery')
('input', 'carrier')
('stockpiling', 'instruments')
('decrease', 'astronomy')
('pediatrics', 'cardiology')
('tools', 'stockpiling')
('golden', 'capsule')
('helicopter', 'fighter')
('juan', 'guan')
('saws', 'props')
('submarine', 'film')
('selective', 'firings')
('supplementary', 'further')
('sale', 'consequences')
('sweet', 'donuts')
('minnesota', 'hill')
('whistleblower', 'panel')
('privatization', 'addressing')
('usable', 'means')
('table', 'interrogation')
('tk', 'caucus')
('pup', 'condor')
('edward', 'boulevard')
('sgm', 'lt')
('expelled', 'quenching')
('northern', 'midway')
('precisely', 'demography')
('instructions', 'case')
('subjects', 'memorandum')
('lo', 'pr')
('without', 'into')
('expenditure', 'exchange')
('sodium', 'grams')
('hostile', 'becoming')
('supplementing', 'attaches')
('symbol

In [37]:
print(len(positive_synonyms))

44062


In [38]:
final_syn_df = pd.DataFrame(positive_synonyms, columns = ["First Term", "Second Term"])

In [40]:
final_syn_df.count()

First Term     44062
Second Term    44062
dtype: int64

In [41]:
final_syn_df.to_csv("final_syns1.csv")